In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import re
#from IPython.display import display, HTML
import HTMLParser

proj_link_list = []
proj_title_list = []
proj_blurb_list = []
proj_by_list = []

# Range(200) because each page has 20 projects, thus -> 200*20=4000 projects
for num in range(200):

    link = urllib.urlopen('https://www.kickstarter.com/discover/advanced?state=successful&sort=most_backed&seed=2462842&page='+str(num+1)).read()
    soup = BeautifulSoup(link, 'html.parser')

    proj_title = soup.find_all("div", class_="project-profile-title text-truncate-xs")
    # project link
    title_str = HTMLParser.HTMLParser().unescape(str(proj_title)) # remove the &quot
    proj_title_list += re.findall('target="">(.*?)</a>',title_str)
    proj_link_list += re.findall('href="(/projects/.*?)[\?]ref=most_backed"', title_str)
    # project blurb
    proj_blurb = soup.find_all("p", class_="project-profile-blurb type-12")
    proj_blurb_list += [s.string for s in proj_blurb]
    proj_by = soup.find_all("p", class_="project-profile-byline type-12")
    proj_by_str = HTMLParser.HTMLParser().unescape(str(proj_by))
    proj_by_list += re.findall('by (.*?) and \d+[,\d+]* backers',proj_by_str)
    
print type(proj_link_list), len(proj_link_list)

<type 'list'> 4000


In [2]:
print proj_link_list[0:10]

['/projects/elanlee/exploding-kittens', '/projects/antsylabs/fidget-cube-a-vinyl-desk-toy', '/projects/readingrainbow/bring-reading-rainbow-back-for-every-child-everywh', '/projects/559914737/the-veronica-mars-movie-project', '/projects/doublefine/double-fine-adventure', '/projects/elanlee/bears-vs-babies-a-card-game', '/projects/getpebble/pebble-time-awesome-smartwatch-no-compromises', '/projects/inxile/torment-tides-of-numenera', '/projects/obsidian/project-eternity', '/projects/playtonic/yooka-laylee-a-3d-platformer-rare-vival']


In [50]:
numBackers_list = []
# currency_type_list = []
amt_pledged_list = []
goal_list = []
location_list = []
category_list = []
pledge_tier_list = []
numBackers_tier_list = []


for url in proj_link_list:

    url = 'https://www.kickstarter.com'+url
    link = urllib.urlopen(url).read()
    soup = BeautifulSoup(link,'html.parser')

    # project stats
    stat = soup.find("div", class_="NS_campaigns__spotlight_stats")
    stat_str = HTMLParser.HTMLParser().unescape(str(stat))
    
    # number of backers
    numBackers = re.findall('<b>(.*?) backers</b>',stat_str)
    numBackers_list.append(int(re.sub(',','',numBackers[0])))
    
    # currency type
    # currency_type = re.findall('<span class="money (.*?) [no\-code]*">', stat_str)
    # currency_type_list.append(currency_type[0])
    
    # amount pledged
    # TODO: DELETE -> amt_pledged = re.findall(' [no\-code]*">\D+(.*?)</span>', stat_str)
    amt_pledged = re.findall('<span class="money">\D+(.*?)</span>', stat_str)
    amt_pledged_list.append(float(re.sub(',','',amt_pledged[0])))

    # project goal
    goal = soup.find("div", class_="col-right col-4 py3 border-left")
    goal_str = HTMLParser.HTMLParser().unescape(str(goal))
    goal = re.findall('>\D+(.*?)</span> goal',goal_str)
    goal_list.append(float(re.sub(',','',goal[0])))

    # project location and category
    temp = soup.find("div", class_="NS_projects__category_location ratio-16-9")
    temp = str(temp)
    location = re.findall('<span aria-hidden="true" class="ksr-icon__location"></span>\n(.*?)\n</a>',temp)
    location_list.append(location[0])
    
    category = re.findall('<span aria-hidden="true" class="ksr-icon__tag"></span>\n(.*?)\n</a>',temp)
    category_list.append(category[0])

    # pledge tiers and corresponding number of backers
    pledge_tier = []
    numBackers_tier = []
    pledge_info = soup.find_all("div", class_="pledge__info")

    for i_pledge in pledge_info:
        pledge_str = str(i_pledge)
        amt = re.findall('About <span>\D(.*?)</span>',pledge_str)
        pledge_tier.append(float(re.sub(',','',amt[0])))
        backers = re.findall('pledge__backer-count">\n(.*?) backer',pledge_str)
        numBackers_tier.append(int(re.sub(',','',backers[0])))

    pledge_tier_list.append(pledge_tier)
    numBackers_tier_list.append(numBackers_tier)
    
df = pd.DataFrame({'by':proj_by_list, 'title':proj_title_list, 'url':proj_link_list, \
                   'blurb':proj_blurb_list, 'num.backers':numBackers_list, \
                   'amt.pledged':amt_pledged_list,\
                   'goal':goal_list,'location':location_list,'category':category_list,\
                   'pledge.tier':pledge_tier_list,'num.backers.tier':numBackers_tier_list})

df.info()
display(df)
df.to_csv('most_backed_2.csv',encoding = 'UTF-8')

['219,382']
['8,782,571']
['154,926']
['6,465,690']
['105,857']
['5,408,916']
['91,585']
['5,702,153']
['87,142']
['3,336,371']
['85,581']
['3,215,679']
['78,471']
['20,338,986']
['74,405']
['4,188,927']
['73,986']
['3,986,929']
['73,206']
['2,090,104']
['70,122']
['3,007,370']
['69,320']
['6,333,295']
['68,929']
['10,266,845']
['67,226']
['3,845,170']
['66,673']
['12,779,843']
['64,867']
['5,545,991']
['63,758']
['3,246,588']
['63,416']
['8,596,474']
['62,642']
['13,285,226']
['61,290']
['2,933,252']
['48,270']
['5,764,229']
['46,520']
['3,105,473']
['44,949']
['9,192,055']
['44,162']
['2,229,344']
['42,713']
['2,032,434']
['41,733']
['2,785,537']
['39,560']
['2,071,927']
['36,863']
['1,022,120']
['36,781']
['1,731,465']
['36,276']
['1,836,447']
['35,550']
['1,076,960']
['35,384']
['1,106,371']
['34,565']
['1,479,046']
['34,397']
['2,134,374']
['33,741']
['1,519,680']
['31,774']
['1,229,015']
['31,497']
['1,204,726']
['31,178']
['3,771,474']
['30,546']
['1,353,024']
['28,818']
['695,1

IOError: [Errno socket error] [Errno 8] nodename nor servname provided, or not known

In [59]:
something = pd.read_csv('Kickstarter.csv')

In [65]:
print something.columns

Index([u'id', u'photo', u'name', u'blurb', u'goal', u'pledged', u'state',
       u'slug', u'disable_communication', u'country', u'currency',
       u'currency_symbol', u'currency_trailing_code', u'deadline',
       u'state_changed_at', u'created_at', u'launched_at', u'staff_pick',
       u'backers_count', u'static_usd_rate', u'usd_pledged', u'creator',
       u'location', u'category', u'profile', u'spotlight', u'urls',
       u'source_url', u'friends', u'is_starred', u'is_backing',
       u'permissions'],
      dtype='object')


In [70]:
print something['state'].value_counts()

successful    2008
failed        1890
canceled       209
live            33
suspended       10
Name: state, dtype: int64


In [ ]:
print "Check length: ", len(proj_by_list), len(proj_title_list), len(proj_link_list), len(proj_blurb_list), \
                        len(numBackers_list), len(amt_pledged_list), len(goal_list), len(location_list), \
                        len(category_list), len(pledge_tier_list), len(numBackers_tier_list)